### Multi-camera recording to CSV 128px

In [4]:
import cv2
import numpy as np
import pandas as pd
import glob

#### Set The Variables Below

In [5]:
recording_dir = '../data/data-butterfly/recordings_2024-03-07_22h35m/' # _RECORDING_DIR_
amount_of_pixels = '128px' # _AMOUNT OF PIXELS_
output_file_location = '../data/data-butterfly/' # _OUTPUT_FILE_LOCATION_
output_file_name = 'red_ball_coordinates_128px_2024-03-07_22h35m.csv' # _OUTPUT_FILE_NAME_

#### Create the CSV file from the multi-camera recording

In [6]:
mp4_files = glob.glob(recording_dir + '*.mp4')

if not mp4_files:
    raise ValueError('No mp4 files found in the recording directory')

df = pd.DataFrame()

lower_red = np.array([0, 0, 100], dtype=np.uint8)
upper_red = np.array([75, 75, 255], dtype=np.uint8)

i=0

for video in mp4_files:
        
    cap = cv2.VideoCapture(video)
    median_x = []
    median_y = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # create a mask for the red pixels in the frame
        mask = cv2.inRange(frame, lower_red, upper_red)
        # apply the mask to the frame so that only the red pixels are shown
        detected_ball = cv2.bitwise_and(frame, frame, mask=mask)    
        # get all the coordinates of red pixels in the frame
        red_coordinates = np.column_stack(np.where((frame[:, :, 0] >= lower_red[0]) & (frame[:, :, 0] <= upper_red[0]) &
        (frame[:, :, 1] >= lower_red[1]) & (frame[:, :, 1] <= upper_red[1]) &
        (frame[:, :, 2] >= lower_red[2]) & (frame[:, :, 2] <= upper_red[2])
        ))    
        # flip the y coordinates to match the origin at the bottom left
        red_coordinates[:, 0] = frame.shape[0] - red_coordinates[:, 0]
        if red_coordinates.size == 0:
            # add NaN if no red ball is detected
            median_x.append(np.nan)
            median_y.append(np.nan)
        else:
            median_x.append(np.median(red_coordinates[:, 1]))
            median_y.append(np.median(red_coordinates[:, 0]))
    
    # add the median x and y coordinates to the dataframe
    df[f'u{i}'] = median_x
    df[f'v{i}'] = median_y
    cap.release()
    
    i+=1

# Save the dataframe to a CSV file
df.to_csv(output_file_location + output_file_name , index=False)